In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from scipy import ndimage
from PIL import Image
%matplotlib inline

In [19]:
# Load the training and testing data and convert it to a PyTorch Tensor
train_dataset = h5py.File("data/train_catvnoncat.h5",'r')
test_dataset = h5py.File("data/test_catvnoncat.h5",'r')

# Separate features(x) and labels(y) for training set
train_set_x_orig = np.array(train_dataset['train_set_x'])
train_set_y_orig = np.array(train_dataset['train_set_y'])

# Convert the features(x) and labels(y) for training data to Pytorch Tensors
train_set_x_orig = torch.from_numpy(train_set_x_orig)
train_set_y_orig = torch.from_numpy(train_set_y_orig)

# Seperate features(x) and labels(y) for the test data
test_set_x_orig = np.array(test_dataset['test_set_x'])
test_set_y_orig = np.array(test_dataset['test_set_y'])

# Convert the features(x) and labels(y) for test data to Pytorch Tensors
test_set_x_orig = torch.from_numpy(test_set_x_orig)
test_set_y_orig = torch.from_numpy(test_set_y_orig)

# Classes
classes = np.array(test_dataset["list_classes"][:])

In [36]:
(train_set_x_orig).shape

torch.Size([209, 64, 64, 3])

In [6]:
type(test_set_x_orig)

torch.Tensor

In [22]:
class TwoLayerNetwork(nn.Module):
    
    """
    n_x: The number of neurons in the input layer
    n_h: The number of neurons in the hidden layer
    n_y: The number of neurons in the output layer
    
    """
    def __init__(self,n_x = 12288, n_h = 7, n_y = 1):
        
        super().__init__()
        
        self.fc1 = nn.Linear(n_x,n_h)
        self.fc2 = nn.Linear(n_h,n_y)
    
    """
    Function for computing the forward pass with a relu activation unit in fully connected layer 1
    and returns an output from the sigmoid activation unit
    """
    def forward(self, X):
        
        X = f.relu(self.fc1(X))
        X = self.fc2(X)
        
        return f.sigmoid(X)

In [30]:
torch.manual_seed(101)
model = TwoLayerNetwork()
model

TwoLayerNetwork(
  (fc1): Linear(in_features=12288, out_features=7, bias=True)
  (fc2): Linear(in_features=7, out_features=1, bias=True)
)

In [31]:
# We define our optimizer as a Cross Entropy Loss Function 
optimizer = nn.CrossEntropyLoss()

# We define our criterion to be Gradient Descent 
criterion = torch.optim.SGD(model.parameters(),lr = 0.001)

In [35]:
# Training our model
import time

start_time  = time.time()

epochs = 10

# Trackers
train_correct = [] # What we get correct during training phase
train_losses = [] # What we get wrong during the training phase
test_correct = [] # what we get correct during the testing phase
test_losses = [] # What we get wrong during the testing phase

for i in range(epochs):
    
    train_corr = 0
    test_corr = 0
    
    for j, (x_train,y_train) in enumerate(train_set_x_orig , train_set_y_orig):
        
        j+=1
        
        y_prediction = model(x_train.view(12288,-1))
        loss = criterion(y_prediction,y_train)
        
        predicted = y_prediction.data
        batch_correct = 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Calculates the time elapsed
total_time = time.time() - start_time 

b'non-cat'